In [1]:
# Import the datafile
import requests
data_url = 'https://raw.githubusercontent.com/eadka/fridgechef/main/Data/RecipeData.json'
data_response = requests.get(data_url)
recipes_data = data_response.json()

In [2]:
# Ensuring all the data has strings because minsearch, under the hood uses TfidfVectorizer and expects each text_field to be a string
for recipe in recipes_data:
    for field in ["dish_name",  "cuisine",  "diet", "tags",  "main_ingredients", 
                 "cooking_time_minutes", "difficulty",  "ingredients_full", 
                 "instructions", "substitutions", "flavor_notes"]:
        value = recipe.get(field, "")
        if isinstance(value,list):
            recipe[field] = " ".join(map(str,value)) # join the list into string
        elif not isinstance(value, str):
            recipe[field] = str(value) # convert numbers to string

In [3]:
# Search engine and indexing
import minsearch

# Indexing the document
index = minsearch.Index(
    text_fields=["dish_name",  "cuisine",  "diet", "tags",  "main_ingredients", 
                 "cooking_time_minutes", "difficulty",  "ingredients_full", 
                 "instructions", "substitutions", "flavor_notes"],
    keyword_fields=[]
)

In [4]:
index.fit(recipes_data)

In [15]:
query = 'Give me recipes for carrots and beans'

In [6]:
index.search(query,num_results=2)

[{'dish_name': 'Rajma Masala',
  'cuisine': 'Indian',
  'diet': 'Vegan',
  'tags': 'protein-rich curry comfort food',
  'main_ingredients': 'kidney beans onion tomato ginger garam masala',
  'cooking_time_minutes': '45',
  'difficulty': 'Medium',
  'ingredients_full': "{'item': 'kidney beans', 'quantity': '2 cups cooked'} {'item': 'onion', 'quantity': '1 large'} {'item': 'tomato', 'quantity': '2'} {'item': 'ginger', 'quantity': '1 inch'} {'item': 'garam masala', 'quantity': '1 tsp'}",
  'instructions': 'Sauté onion and ginger until golden. Add tomato and spices, cook until soft. Add kidney beans, simmer for 20 minutes.',
  'substitutions': "{'kidney beans': ['black beans']}",
  'flavor_notes': 'Rich, spiced, and hearty.'},
 {'dish_name': 'Minestrone Soup',
  'cuisine': 'Italian',
  'diet': 'Vegan',
  'tags': 'soup hearty vegetable-rich',
  'main_ingredients': 'carrot celery zucchini beans pasta tomato',
  'cooking_time_minutes': '40',
  'difficulty': 'Easy',
  'ingredients_full': "{'it

In [7]:
# Open AI for LLM integration
from openai import OpenAI

client = OpenAI()

In [17]:
# response = client.chat.completions.create(
#     model='gpt-4o-mini',
#     messages=[{"role": "user", "content": query}]
# )

# response.choices[0].message.content

'Here are three delicious recipes that feature carrots and beans as key ingredients:\n\n### 1. **Carrot and Bean Stir-Fry**\n\n**Ingredients:**\n- 2 cups green beans, trimmed\n- 2 medium carrots, sliced thinly\n- 1 tablespoon vegetable oil\n- 3 cloves garlic, minced\n- 1 inch ginger, grated\n- 3 tablespoons soy sauce\n- 1 tablespoon sesame oil\n- Sesame seeds for garnish\n- Salt and pepper to taste\n\n**Instructions:**\n1. Heat the vegetable oil in a large skillet or wok over medium-high heat.\n2. Add the minced garlic and grated ginger. Sauté for about 30 seconds until fragrant.\n3. Add the sliced carrots and green beans. Stir-fry for about 5-7 minutes until the vegetables are tender but still crisp.\n4. Pour in the soy sauce and sesame oil. Toss to coat the vegetables evenly.\n5. Season with salt and pepper to taste. Cook for another minute.\n6. Remove from heat and garnish with sesame seeds before serving. Enjoy it as a side dish or over cooked rice or quinoa.\n\n### 2. **Carrot and

In [8]:
# Defining the RAG flow
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
recipes_data[0]

{'dish_name': 'Vegetable Pad Thai',
 'cuisine': 'Thai',
 'diet': 'Vegan',
 'tags': 'quick noodles stir-fry',
 'main_ingredients': 'rice noodles tofu carrot bean sprouts spring onions peanuts soy sauce lime garlic',
 'cooking_time_minutes': '25',
 'difficulty': 'Easy',
 'ingredients_full': "{'item': 'rice noodles', 'quantity': '200g'} {'item': 'tofu', 'quantity': '150g'} {'item': 'carrot', 'quantity': '1 medium'} {'item': 'bean sprouts', 'quantity': '1 cup'} {'item': 'spring onions', 'quantity': '2'} {'item': 'peanuts', 'quantity': '2 tbsp, crushed'} {'item': 'soy sauce', 'quantity': '3 tbsp'} {'item': 'lime', 'quantity': '1'} {'item': 'garlic', 'quantity': '2 cloves'}",
 'instructions': 'Soak rice noodles in warm water for 20 minutes. Stir-fry garlic and tofu until golden. Add vegetables and stir-fry for 2-3 minutes. Add noodles and sauce, toss until combined. Garnish with peanuts and lime.',
 'substitutions': "{'tofu': ['tempeh', 'chickpeas'], 'soy sauce': ['tamari', 'coconut aminos']

In [16]:
prompt_template = """
You're a "Fridge Chef", a helpful cooking assistant. 
The user will give you a list of vegetables or ingredients they have available.
You must recommend vegetarian or vegan dishes from the CONTEXT, which is the recipes from the database.
Base your answer only on the recipes in the CONTEXT.
If you cannot find an exact match, suggest the closest dishes using the available ingredients.

When answering:
- Include the dish name, cuisine, diet type, main ingredients, and cooking time.
- Provide short cooking instructions based on the CONTEXT.
- Suggest possible ingredient substitutions if given in the CONTEXT.
- If multiple dishes fit, return the top 3–5 most relevant recipes.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

entry_template = """
dish_name: {dish_name}
cuisine: {cuisine}
diet: {diet}
tags: {tags}
main_ingredients: {main_ingredients}
cooking_time_minutes: {cooking_time_minutes}
difficulty: {difficulty}
ingredients_full: {ingredients_full}
instructions: {instructions}
substitutions: {substitutions}
flavor_notes: {flavor_notes}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [18]:
print(prompt)

You're a "Fridge Chef", a helpful cooking assistant. 
The user will give you a list of vegetables or ingredients they have available.
You must recommend vegetarian or vegan dishes from the CONTEXT, which is the recipes from the database.
Base your answer only on the recipes in the CONTEXT.
If you cannot find an exact match, suggest the closest dishes using the available ingredients.

When answering:
- Include the dish name, cuisine, diet type, main ingredients, and cooking time.
- Provide short cooking instructions based on the CONTEXT.
- Suggest possible ingredient substitutions if given in the CONTEXT.
- If multiple dishes fit, return the top 3–5 most relevant recipes.

QUESTION: Give me recipes for carrots and beans

CONTEXT: 
dish_name: Rajma Masala
cuisine: Indian
diet: Vegan
tags: protein-rich curry comfort food
main_ingredients: kidney beans onion tomato ginger garam masala
cooking_time_minutes: 45
difficulty: Medium
ingredients_full: {'item': 'kidney beans', 'quantity': '2 cups

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
rag(query)

'Here are some recipes you can make with carrots and beans:\n\n1. **Minestrone Soup**\n   - **Cuisine:** Italian\n   - **Diet:** Vegan\n   - **Main Ingredients:** Carrot, celery, zucchini, cannellini beans, pasta, tomato.\n   - **Cooking Time:** 40 minutes\n   - **Cooking Instructions:** \n     1. Sauté chopped carrots and celery in a pot for 5 minutes.\n     2. Add zucchini, chopped tomatoes, beans, and vegetable broth. \n     3. Simmer for 20 minutes, then add pasta and cook until tender.\n   - **Substitutions:** Use kidney beans instead of cannellini beans or replace pasta with rice noodles.\n\n2. **Rajma Masala**\n   - **Cuisine:** Indian\n   - **Diet:** Vegan\n   - **Main Ingredients:** Kidney beans, onion, tomato, ginger, garam masala.\n   - **Cooking Time:** 45 minutes\n   - **Cooking Instructions:** \n     1. Sauté onion and ginger until golden.\n     2. Add chopped tomatoes and spices, cooking until soft. \n     3. Add kidney beans and simmer for 20 minutes.\n   - **Substituti

In [22]:
answer = rag('I have nothing in my fridge, what do you suggest?')
print(answer)

Since you have nothing in your fridge, it's tricky to suggest exact recipes. However, I can guide you on what to look for or purchase to make a delicious meal. Here are some vegetarian or vegan dishes that can be made with basic ingredients common in most kitchens:

1. **Mushroom Stroganoff**
   - **Cuisine:** Italian
   - **Diet:** Vegetarian
   - **Main Ingredients:** Mushrooms, onion, garlic, cream, pasta
   - **Cooking Time:** 30 minutes
   - **Instructions:** Sauté onions and garlic in a pan, add sliced mushrooms, and cook until soft. Pour in cream and let it simmer for about 5 minutes. Toss with cooked pasta before serving.
   - **Possible Substitutions:** If you’re dairy-free, you can use coconut cream or cashew cream instead of regular cream.

2. **Gnocchi with Tomato Sauce**
   - **Cuisine:** Italian
   - **Diet:** Vegetarian
   - **Main Ingredients:** Gnocchi, tomatoes, garlic, basil
   - **Cooking Time:** 25 minutes
   - **Instructions:** Cook gnocchi in boiling water until 